In [8]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from binance.client import Client
from binance import ThreadedWebsocketManager
import pandas as pd
from datetime import datetime, timedelta
import time
from dateutil import parser
plt.style.use('seaborn')
import seaborn as sns

In [4]:
class LongShortTrader():
    def __init__(self, symbol, bar_length, return_thresh, volume_thresh, units, position = 0):
        self.symbol = symbol
        self.bar_length = bar_length
        self.api = 'UvWmZSuzVt7WQTGGRmu1AEW7oDy5Aij8H5yaitGIfd2XbRdiKQX71YUpypYgCaOG'
        self.secret = 'PebaWkuND7ae2zDXXadlpXkooFvoprzY4iJkXCfsgu3HrQR5nYKvaU1q7cpylLLI'
        self.client = Client(api_key = self.api, api_secret = self.secret, tld = "com", testnet = True)
        self.data = pd.DataFrame(columns = ['Open', 'High', 'Low', 'Close', 'Volume', 'Complete'])
        self.available_intervals = ['1m', '3m', '5m', '15m', '30m', '1h', '2h', '6h', '8h', '10h', '12h', '1d', '3d', '1w', '1M']
        self.units = units
        self.position = position
        self.trades = 0
        self.trade_values = []
        
        ################ Atributos específicos de la clase Back Test ###############
        
        self.return_thresholds = return_thresh
        self.volume_thresholds = volume_thresh
    
        
    def start_trading(self, historical_days):
        
        self.twm = ThreadedWebsocketManager()
        self.twm.start()
        
        if self.bar_length in self.available_intervals:
            self.get_most_recent(symbol = self.symbol, interval = self.bar_length, days = historical_days)
            self.twm.start_kline_socket(callback = self.stream_candles, symbol = self.symbol, interval = self.bar_length)
        else:
            print('Intervalo no es posible, favor ingresar un intervalo válido. ')
            
    def get_most_recent(self,symbol, interval, days):
    
        now = datetime.utcnow()
        past = int(datetime.strptime((now-timedelta(days=days)).strftime('%Y-%m-%d'), "%Y-%m-%d").timestamp()*1000)

        bars = self.client.get_historical_klines(symbol = symbol, interval = interval,
                                            start_str = past, end_str = None, limit = 1000)

        df = pd.DataFrame(bars)
        df['Date'] = pd.to_datetime(df.iloc[:,0], unit = 'ms')
        df.columns = ['Open Time', 'Open', 'High', 'Low', 'Close','Volume',
                      'Close Time', 'Quote Asset Volume', 'Number of Trades',
                      'Tracker Buy Base Asset Volume','Taker Buy Quote Asset Volume', 'Ignore', 'Date']
        df = df[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']].copy()
        df.set_index("Date", inplace = True)
        for column in df.columns:
            df[column] = pd.to_numeric(df[column], errors = 'coerce')
        df['Complete'] = [True for row in range(len(df) - 1)] + [False]
        
        print(df.tail())
        self.data = df
        
    def stream_candles(self, msg):
        #extract the required items from msg
        event_time = pd.to_datetime(msg['E'], unit = 'ms')
        start_time = pd.to_datetime(msg['k']['t'], unit = 'ms')
        first    = float(msg['k']['o'])
        high     = float(msg['k']['h'])
        low      = float(msg['k']['l'])
        close    = float(msg['k']['c'])
        volume   = float(msg['k']['v'])
        complete = msg['k']['x']
        
        #stop trading
        
        if self.trades >= 5:
            self.twm.stop()
            if self.position == 1:
                order = self.client.create_order(symbol = self.symbol, side = 'SELL', type = 'MARKET', quantity = self.units)
                self.report_trade(order, 'GOING NEUTRAL AND STOP')
                self.position = 0
            elif self.position == -1:
                order = self.client.create_order(symbol = self.symbol, side = 'BUY', type = 'MARKET', quantity = self.units)
                self.report_trade(order, 'GOING NEUTRAL AND STOP')
                self.position = 0
            else:
                print('STOP')

        #print out
        print('.', end = "", flush = True) # just print something to get a feedback (everything OK)
        
        #Example how to stop trading session
#         if event_time >= datetime(2022,8,11,9,55) 8th August 2022, 9 55 pm
#             self.twm.stop()
#             if self.postion != 0:
#                 order = client.create_order(symbol = self.symbol, side = 'SELL', type = 'MARKET', quantity = self.units)
#                 self.report_trade(order, 'GOING NEUTRAL AND STOP')
#                 self.position = 0
#             else:
#                 print(STOP)

        #More stop examples
        #if self.trades >= xyz
        #if self.cum_profits <> xyz

        #feed df (add new bar / update latest bar )
        self.data.loc[start_time] = [first, high, low, close, volume, complete]
        
        # prepare  features and define strategy / trading positions whenever the latest bar is complete
        if complete == True:
            self.define_strategy()
            self.execute_trades()
            
    def define_strategy(self):
        
        data = self.data.copy()
        
        #Define your strategy here
        
        data = data[['Close', 'Volume']].copy()
        data['returns'] = np.log(data.Close / data.Close.shift())
        data['vol_ch'] = np.log(data.Volume.div(data.Volume.shift(1)))
        data.loc[data.vol_ch > 3, 'vol_ch'] = np.nan
        data.loc[data.vol_ch < -3, 'vol_ch'] = np.nan
        #cond2 = df.vol_ch.between(self.volume_thresh[0], self.volume_thresh[1])
        cond1 = data.returns.between(self.return_thresholds[0], self.return_thresholds[1])
        cond2 = data.vol_ch.between(self.volume_thresholds[0], self.volume_thresholds[1])
        cond3 = data.returns.between(self.return_thresholds[2], self.return_thresholds[3])
        cond4 = data.vol_ch.between(self.volume_thresholds[2], self.volume_thresholds[3])
        
        data['position'] = 0
        data.loc[cond1 & cond2, 'position'] = -1
        data.loc[cond3 & cond4, 'position'] = 1
        
        #prepare data
        
        print(data.tail())
        self.prepared_data = data.copy()
        
    def execute_trades(self):
        print(self.prepared_data['position'].iloc[-1])
        print(self.position)
        self.position = 1
        if self.prepared_data['position'].iloc[-1] == 1: #if position is long -> go/stay long
            if self.position == 0:
                order = self.client.create_order(symbol = self.symbol, side='BUY', type = 'MARKET', quantity = self.units)
                self.report_trade(order, 'GOING LONG')
            elif self.position == -1:
                order = self.client.create_order(symbol = self.symbol, side='BUY', type = 'MARKET', quantity = self.units)
                self.report_trade(order, 'GOING NEUTRAL')
                time.sleep(0.1)
                order = self.client.create_order(symbol = self.symbol, side='BUY', type = 'MARKET', quantity = self.units)
                self.report_trade(order, 'GOING LONG')
            self.position = 1
        elif self.prepared_data['position'].iloc[-1] == 0: #if position neutral  -> go/stay neutral
            if self.position == 1:
                order = self.client.create_order(symbol = self.symbol, side='SELL', type = 'MARKET', quantity = self.units)
                self.report_trade(order, 'GOING NEUTRAL IN ZERO FROM 0')
            elif self.position == -1:
                order = self.client.create_order(symbol = self.symbol, side='BUY', type = 'MARKET', quantity = self.units)
                self.report_trade(order, 'GOING NEUTRAL')
            self.position = 0
        elif self.prepared_data['position'].iloc[-1] == -1:  #if position is short  ->  go/stay short
            if self.position == 0:
                order = self.client.create_order(symbol = self.symbol, side='SELL', type = 'MARKET', quantity = self.units)
                self.report_trade(order, 'GOING SHORT')
            elif self.position == 1:
                order = self.client.create_order(symbol = self.symbol, side='SELL', type = 'MARKET', quantity = self.units)
                self.report_trade(order, 'GOING NEUTRAL IN -1 FROM 1')
                time.sleep(0.1)
                order = self.client.create_order(symbol = self.symbol, side='SELL', type = 'MARKET', quantity = self.units)
                self.report_trade(order, 'GOING SHORT')
            self.position = -1
        print('saliendo de metodo execute trades')
            
    def report_trade(self, order, going):
        print('estoy entrando a dividir')
        #extract data from order object
        side = order['side']
        time = pd.to_datetime(order['transactTime'], unit = 'ms')
        base_units = float(order['executedQty'])
        quote_units = float(order['cummulativeQuoteQty'])
        price = round(quote_units / base_units, 5)
        
        #calculate trading profits
        self.trades +=1
        if side == 'BUY':
            self.trade_values.append(-quote_units)
        elif side == 'SELL':
            self.trade_values.append(quote_units)
            
        if self.trades % 2 == 0:
            real_profit = round(np.sum(self.trade_values[-2:]), 3) #[-2:] sume solamente los dos ultimos
            cum_profits = round(np.sum(self.trade_values), 3)
        else:
            real_profit = 0
            cum_profits = round(np.sum(self.trade_values[:-1]), 3) #[:-1] sume todos menos el ultimo
            
        #print trade report
        print(2 * "\n" + 100* "-")
        print("{} | {}".format(time, going))
        print("{} | Base_Units = {} | Quote_Units = {} | Price = {}".format(time, base_units, quote_units, price))
        print("{} | Profit = {} | CumProfits = {} ".format(time, real_profit, cum_profits))
        print(100 * "-" + "\n")

In [5]:
symbol = 'ETHUSDT'
bar_length = '1m'
return_thresh = [-2.04208520e-03, -1.15948325e-03, -1.91641325e-04,  6.41910357e-06]
volume_thresh = [-1.44201736, -0.9517655 , -0.21242835, -0.03758546]
units = 0.01

In [6]:
test = LongShortTrader(symbol= symbol, bar_length = bar_length, return_thresh = return_thresh
                                 , volume_thresh = volume_thresh, units = units)

In [7]:
test.start_trading(7)

                        Open     High      Low    Close    Volume  Complete
Date                                                                       
2022-11-13 19:58:00  1228.09  1229.90  1228.08  1229.37  18.77417      True
2022-11-13 19:59:00  1229.36  1229.52  1228.82  1229.11  11.30005      True
2022-11-13 20:00:00  1229.04  1229.04  1227.98  1228.89  12.59886      True
2022-11-13 20:01:00  1228.75  1228.89  1226.25  1227.47  16.77061      True
2022-11-13 20:02:00  1227.47  1228.92  1227.21  1228.64  10.16192     False


CANCEL read_loop
Task exception was never retrieved
future: <Task finished name='Task-8' coro=<ThreadedApiManager.start_listener() done, defined at C:\Users\maiky.rodriguez\Anaconda3\lib\site-packages\binance\threaded_stream.py:42> exception=RuntimeError("Task <Task pending name='Task-14' coro=<Queue.get() running at C:\\Users\\maiky.rodriguez\\Anaconda3\\lib\\asyncio\\queues.py:166> cb=[_release_waiter(<Future pendi...20B0DE070>()]>)() at C:\\Users\\maiky.rodriguez\\Anaconda3\\lib\\asyncio\\tasks.py:416]> got Future <Future pending> attached to a different loop")>
Traceback (most recent call last):
  File "C:\Users\maiky.rodriguez\Anaconda3\lib\site-packages\binance\threaded_stream.py", line 46, in start_listener
    msg = await asyncio.wait_for(s.recv(), 3)
  File "C:\Users\maiky.rodriguez\Anaconda3\lib\asyncio\tasks.py", line 479, in wait_for
    return fut.result()
  File "C:\Users\maiky.rodriguez\Anaconda3\lib\site-packages\binance\streams.py", line 197, in recv
    res = await as